In [ ]:
!pip install altair pandas vega_datasets selenium

In [2]:
!npm install -g vega-lite vega-cli canvas

/usr/bin/bash: line 1: npm: command not found


In [1]:
import altair_saver
import altair_viewer

print("altair_saver:", altair_saver.__version__)
print("altair_viewer:", altair_viewer.__version__)

altair_saver: 0.5.0
altair_viewer: 0.4.0


In [13]:
import altair as alt
import pandas as pd

df = pd.read_csv('./results/benchmarking_res_final.csv')

metrics = ['Recall', 'F1']
colors = ["#1874CD", "#FF6F61"]

df_long = df.melt(id_vars='model', value_vars=metrics,
                  var_name='metric', value_name='score')

df_long['score_str'] = df_long['score'].apply(lambda x: f"{x:.2f}".lstrip('0'))

base = alt.Chart(df_long).encode(
    y=alt.Y('model:N', axis=alt.Axis(labelFontSize=40, title=None, labelLimit=500, labelPadding=10)),
    yOffset='metric:N'
)

bars = base.mark_bar(cornerRadiusEnd=10, opacity=0.8).encode(
    x=alt.X('score:Q', axis=alt.Axis(labelFontSize=35, title=None, grid=True, domain=True, tickCount=5)),
    color=alt.Color('metric:N', scale=alt.Scale(domain=metrics, range=colors),
                    legend=alt.Legend(title=None, labelFontSize=40, orient='top'))
).properties(
    width=700,
    height=70 * len(df['model'].unique()),
    # Note: no padding here
)

text = base.mark_text(
    align='left',
    baseline='middle',
    dx=3,
    font='Liberation Sans',
    fontSize=34,
    color='black'
).encode(
    x='score:Q',
    text=alt.Text('score_str:N')
)

# Layer the bars and text first
chart = bars + text

# Now set padding on the layered chart
chart = chart.properties(
    padding={"left": 50, "top": 50, "right": 50, "bottom": 50}
)

# Configure as before
chart = chart.configure_view(stroke=None)\
    .configure_axis(labelFontSize=40, titleFontSize=30)\
    .configure_legend(labelFontSize=40, symbolSize=400, orient='top')\
    .configure_axisLeft(labelLimit=400)


In [14]:
import vl_convert as vlc
chart_json = chart.to_json()
svg_str = vlc.vegalite_to_svg(chart_json)

with open("benchmarking.svg", "wt") as f:
    f.write(svg_str)